<a href="https://colab.research.google.com/github/rahiakela/transformers-for-natural-language-processing/blob/main/1-model-architecture-of-the-transformer/1_positional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Positional encoding

The Transformer's subsequent layers do not start empty-handed. They have learned word embeddings that already provide information on how the words can be associated.

However, a big chunk of information is missing because no additional vector or information indicates a word's position in a sequence.

The designers of the Transformer came up with yet another innovative feature: positional encoding.

Let's see how positional encoding works.

We enter this positional encoding function of the Transformer with no idea of the position of a word in a sequence:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/position-encoding.png?raw=1' width='800'/>

We cannot create independent positional vectors that would have a high cost on the training speed of the Transformer and make attention sub-layers very complex to work with. 

**The idea is to add a positional encoding value to the input embedding instead of having additional vectors to describe the position of a token in a sequence.**

We also know that the Transformer expects a fixed size $d_{model} = 512$ (or other constant value for the model) for each vector of the output of the positional encoding function.

## Setup

In [ ]:
!pip install --upgrade gensim

In [ ]:
import torch
import nltk
nltk.download('punkt')

In [ ]:
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 

In [ ]:
!wget https://raw.githubusercontent.com/rahiakela/transformers-for-natural-language-processing/main/1-model-architecture-of-the-transformer/text.txt

## Introduction

If we go back to the sentence we used in the word embedding sub-layer, we can see that **black** and **brown** may be similar, but they are far apart:

```
The black cat sat on the couch and the brown dog slept on the rug.
```

The word **black** is in position 2, pos=2, and the word **brown** is in position 10, pos=10.

Our problem is to find a way to add a value to the word embedding of each word so that it has that information. However, we need to add a value to the $d_{model} = 512$ dimensions! For each word embedding vector, we need to find a way to provide information to $i$ in the $range(0,512)$ dimensions of the word embedding vector of **black** and **brown**.

There are many ways to achieve this goal. The designers found a clever way to use a unit sphere to represent positional encoding with sine and cosine values that will thus remain small but very useful.

Vaswani et al. (2017) provide sine and cosine functions so that we can generate different frequencies for the positional encoding (PE) for each position and each dimension i of the dmodel = 512 of the word embedding vector:

$$ PE_{(pos,2i)}=sin\begin{pmatrix} \frac{pos}{10000\frac{2i}{d_{model}}} \end{pmatrix}$$

$$ PE_{(pos,2i+1)}=cos\begin{pmatrix} \frac{pos}{10000\frac{2i}{d_{model}}} \end{pmatrix}$$

If we start at the beginning of the word embedding vector, we will begin with a constant (512), $i=0$, and end with $i=511$. **This means that the sine function will be applied to the even numbers and the cosine function to the odd numbers**. Some implementations do it differently. In that case, the domain of the sine function can be $i \in [0, 255]$  and the domain of the cosine function can be $i \in [256, 512]$. This will produce similar results.

A literal translation into Python produces the following code for a positional vector $pe[0][i]$ for a position `pos`: